### First exploration about data

In [32]:
import pandas as pd
import jieba
%matplotlib inline

product_log = pd.read_csv("../data.tsv",sep="\t",names=["product_name", "category", "query", "event", "date"])
print product_log.shape, product_log.drop_duplicates().shape
product_log = product_log.drop_duplicates()
product_log.head(5)

(10317, 5) (10310, 5)


,product_name,category,query,event,date
0,--- X 10 --- 七色 多層次搭配 圓下擺 LAYERED 素面 無袖背心 打底,Male Fashion,無袖,Impression,31/7/17
1,︱IBIT︱Gymshark 熱銷款 運動T恤 健身T恤 圓領短T 運動短T 健身鯊魚,Male Fashion,gymshark,Impression,31/7/17
2,︱IBIT︱Gymshark 超高彈性 短褲 運動短褲 跑步短褲 深蹲褲 訓練短褲,Male Fashion,gymshark,Impression,31/7/17
3,::另類情侶兄弟姊妹殼::電力滿格/不足黑白趣味浮雕手機軟殼i5/i5s/i5se/i6/i...,Mobile & Gadgets,軟殼,Click,31/7/17
4,：新舊手機商場：Iphone6 16金 （需要看細圖密我）,Mobile & Gadgets,iphone6 系列,Impression,30/7/17


In [38]:
product_log[product_log.product_name == "創意雙夾手機懶人支架 懶人夾 床頭手機支架 雙夾支架 6.3吋以下手機都能使用"]

,product_name,category,query,event,date
6431,創意雙夾手機懶人支架 懶人夾 床頭手機支架 雙夾支架 6.3吋以下手機都能使用,Mobile & Gadgets,手機支架 機車,Impression,31/7/17
6432,創意雙夾手機懶人支架 懶人夾 床頭手機支架 雙夾支架 6.3吋以下手機都能使用,Mobile & Gadgets,空機手機,Impression,31/7/17


In [36]:
from collections import Counter
for w, frequency in Counter(product_log.product_name).iteritems():
    if frequency > 1:
        print w, frequency

ⓈSpanforce(現貨)黑色高磅數高品質膝蓋刀片長破褲(0256) 2
【MasCool】adidas Originals Shorts 運動 黑色 棉褲 短褲 三間線 三葉草 AJ6942 2
衝評特價 現貨💯迪士尼 公主系列 愛麗絲 艾莉兒 長髮公主 四邊全包軟殼 金屬按鍵 浮雕手感 小美人魚 2
⭐️NoName⭐️ Nike 大勾棉褲褲 2
stussy聯合辛普森 圓領t恤 情侶t恤 潮牌滑板短T 男生t恤 情侶TEE 同款 2017新款 3
*SILIA*亂步奇譚 手機殼三星S3 S4 S5 S6 S7 EDGE NOTE2 NOTE3 NOTE4 NOTE5 NOTE6 A8 A7 A5 A3 E7 J3 J5 J7 LG G3 G4 G5 OPPO R5 R7 PLUS R9 F1 N3 2
飛岳。Nike 男 黑 白 新款 雙面穿團體球衣 Dri-FIT 籃球服 籃球褲 球褲 套裝 631064012 2
蝦皮最便宜 Under Armour 安德瑪 運動背心 UA 快乾休閒無袖T恤男超彈舒適透氣寬鬆健身跑步籃球運動衣 2
【現貨】 I6/I7 情侶 鏡面 蝙蝠俠 鋼鐵人 手機殼 6s I7plus iphone7 i7+ 088 2
Double AK NIKE NSW SHORT 836278-010 男款 夏季 黑底白勾 休閒短褲 棉質 2
平口連身褲 2
OPPO A77 黑色 (新竹可自取) 2
日韓原宿字母刺繡工藝織帶 雙環金屬皮帶腰帶 高街男女學生褲帶 潮流爆款 多色可選 2
現貨6s迪士尼閉眼美人魚艾莉兒夢幻手機殼iPhone5se/6sp/7p可愛vivox7/x9軟殼oppoR9s 2
Oppo R11 遠傳續約全新機 12000 保固一年 2
【現貨】美圖 MEITU M8 (4G/64G) 5.2吋美少女戰士限量版 空機價 2
《Y-S新賣場》網紅自拍神器 Bcase手機支架底座 自拍不求人 橫豎拍 直播支架手機支架藍芽自拍遙控迷你自拍桿三腳架 2
立體重低音金屬耳機Q9，打敗品牌耳機! 線控耳機旗艦款 線控耳機 earpods 蘋果耳機 耳機 免持耳機 3
蘋果風 原廠正品 手機殼 iPhone7 i6 i7 6s plus 7plus 防摔邊框 背蓋殼 保護殼 防摔殼 2
FINDSENSE MD 時尚 男 韓國 經

### Analyze how the search system work
- Exact match vs semantic match?

In [2]:
def exact_match((title,query)):
    if (query in title) or (query in title.lower()):
        return True
    return False

product_log["exact_match"] = map(exact_match,zip(product_log.product_name, product_log['query']))

In [17]:
from jieba import analyse

In [19]:
i=12
sentence = product_log.product_name.iloc[i]
keyword = product_log['query'].iloc[i]
print sentence

for w in jieba.cut(sentence):
    print w

‼️現貨‼️蘋果/安卓/Type C尼龍金屬25CM充電線 蘋果/安卓/Type C手機數據線 25公分手機充電線
‼
️
現貨
‼
️
蘋果
/
安卓
/
Type
 
C
尼龍
金屬
25CM
充電線
 
蘋果
/
安卓
/
Type
 
C
手機
數據線
 
25
公
分手
機充
電線


In [15]:
print product_log.product_name.iloc[12]
for w in analyse.textrank(product_log.product_name.iloc[12], topK=10, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')):
    print w

‼️現貨‼️蘋果/安卓/Type C尼龍金屬25CM充電線 蘋果/安卓/Type C手機數據線 25公分手機充電線
蘋果
公
充電線
分手
安
數據線
電線
機充
金屬
尼龍


### Start from simple model
- Determine word importance based on frequency only

In [39]:
# preprocessing
def preprocess(string):
    return list(jieba.cut(string))
titles = product_log.product_name.map(preprocess)
keywords = product_log['query'].map(preprocess)

In [50]:
i = 15
for w in titles[i]:
    print w
    
print "-------------"
for w in keywords[i]:
    print w

!
LLM
!
2
個
優惠卡娜
赫拉
軟殼
指環掛
繩
三星
s7
 
NOTE4
 
NOTE5
 
2016J5
 
J7
 
2016J7
手機殼
-------------
卡娜
赫拉
 
手機殼
